# Neutrino direction reconstruction


### Jan 7, 2021

## Code

In [5]:
import sys
import csv
# sys.path.insert(0,"/users/PCON0003/cond0068/.local/lib/python3.7/")
sys.path.append("/users/PAS0654/osu8354/ARA_cvmfs/root_build/lib/") # go to parent dir
sys.path.append("/users/PCON0003/cond0068/.local/lib/python3.7/site-packages/")
# sys.path.append("/users/PCON0003/cond0068/pyrex_sims/fromBen/thesis_work/pyrex-custom/analysis/custom/analysis/")
import ROOT
import math
import numpy as np
from ROOT import TH1D,TF1, gRandom, gPad, gStyle
import matplotlib as mpl
import matplotlib.pyplot as plt
from ROOT import TChain, TSelector, TTree
import os
import matplotlib.colors as mcolors
import scipy
from matplotlib.colors import LogNorm
import pandas as pd
import pyrex
import seaborn as sns
%matplotlib inline
sys.path.insert(1, "/users/PAS0654/osu8354/ARA_cvmfs/source/AraRoot/analysis/ARA_analysis/CenA_sourceSearch/Stokes")
import deDisperse_util as util
from pyrex.internal_functions import normalize
my_path_plots = os.path.abspath("./plots/")
import pyrex.custom.ara as ara


In [6]:
# mpl.use('agg') 
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.unicode'] = True
mpl.rcParams['mathtext.rm'] = 'Times New Roman'
mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'

mpl.rc('font', family='serif', size=12)
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['ytick.major.size'] = 5

mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 18
# mpl.rc('font', size=16)
mpl.rc('axes', titlesize=20)

current_palette = sns.color_palette('colorblind', 10)
import warnings
warnings.filterwarnings("ignore")


In [7]:
def SphericalToCartesian(phi, theta):
    x = np.sin(theta)*np.cos(phi)
    y = np.sin(theta)*np.sin(phi)
    z = np.cos(theta)
    
    return np.array([x,y,z])

## Yet another event

The MC truth neutrino direction (in AraSim Earth-centered coordinates) is 
$$
\hat{v} = (-0.3454016638537625, -0.7187225447988469, -0.6034364872999659)
$$

#### Using MC from AraSim at source

In [8]:
polSrc = np.array([0.123453,0.168241,0.977985])
launchVecSrc = np.array([-0.383979,-0.900658,0.203409])
viewAngleSrc = np.radians(48.9129)

In [9]:
np.sin(viewAngleSrc)*polSrc-np.cos(viewAngleSrc)*launchVecSrc #equal to nnu, module a minus sign (TBD)

array([0.34540107, 0.71872261, 0.6034369 ])

In [10]:
theta = np.arccos(launchVecSrc[2])
# 180-np.degrees(np.arccos(0.28209558))

In [11]:
np.degrees(theta)

78.26362074470669

#### Using output from AraSim at antenna

In [17]:
polVec = np.array([0.059005,0.017102,0.998111])
propVec = np.array([0.39201006, 0.91913715, -0.03892302])
phi = np.arctan2(propVec[1],propVec[0])
theta = np.pi-np.arccos(launchVecSrc[2])
recVec = propVec
propVec = -1*SphericalToCartesian(phi,theta)
viewAngle = 0.8536910375076152
antennaPos = np.array([10004.71607,9989.46621,6359632.39013])
antennaPos = antennaPos/np.linalg.norm(antennaPos)
print("ViewAngle = %0.3f"%np.degrees(viewAngle))

ViewAngle = 48.913


In [13]:
np.sin(viewAngle)*polVec-np.cos(viewAngle)*propVec #Not equal to nnu

array([0.29690918, 0.60477202, 0.618606  ])

In [42]:
recAngle = np.pi-np.arccos(recVec[2])
launchAngle = np.arccos(launchVecSrc[2])
angleChange = recAngle-launchAngle
polOmega = np.arccos(polVec[2])
polPsi = np.arctan2(polVec[1],polVec[0])
newPolOmega =polOmega+angleChange 

In [43]:
SphericalToCartesian(polPsi,newPolOmega)

array([0.2165154 , 0.06275479, 0.97426019])

In [49]:
np.sin(newPolOmega)

0.2254264421625083

#### Enter rotation matrix

In [44]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [52]:
np.dot(polVec,rotation_matrix_from_vectors(polSrc,polVec))

array([0.12345296, 0.16824094, 0.97798466])